# 시드값 고정 및 GPU 장비 설정
## 1.시드값 고정

In [1]:
import torch
import random
import numpy as np
import os

seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

## 2.GPU 장비 설정

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# 데이터 준비

In [3]:
import pandas as pd

data_path = '/kaggle/input/plant-pathology-2020-fgvc7/'
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

## 1.훈련 데이터, 검증 데이터 분리

In [4]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(train, 
                                test_size=0.1, 
                                stratify=train[['healthy', 'multiple_diseases', 'rust', 'scab']], 
                                random_state=50)

print('훈련 데이터 개수:', len(train))
print('검증 데이터 개수:', len(valid))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


훈련 데이터 개수: 1638
검증 데이터 개수: 183


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


## 2.데이터셋 클래스 정의

In [5]:
import cv2
from torch.utils.data import Dataset
import numpy as np

class ImageDataset(Dataset):
    # 초기화 메서드(생성자)
    def __init__(self, df, img_dir='./', transform=None, is_test=False):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test
    
    # 데이터셋 크기 반환 메서드
    def __len__(self):
        return len(self.df)
    
    # 인덱스(idx)에 해당하는 데이터 변환 메서드
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]                  # 이미지 ID
        img_path = self.img_dir + img_id + '.jpg'      # 이미지 파일 경로
        image = cv2.imread(img_path)                   # 이미지 파일 읽기
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # 이미지 색상 보정
        
        if self.transform is not None:
            image = self.transform(image=image)['image']
        if self.is_test:
            return image
        else:
            label = np.argmax(self.df.iloc[idx, 1:5])
            return image, label

## 3.이미지 변환기 정의

In [6]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 훈련 데이터용 변환기
transform_train = A.Compose([
    A.Resize(450, 650),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
    A.VerticalFlip(p=0.2),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=30, p=0.3),
    A.OneOf([A.Emboss(p=1), A.Sharpen(p=1), A.Blur(p=1)], p=0.3),
    A.PiecewiseAffine(p=0.3),
    A.Normalize(),
    ToTensorV2()
])

In [7]:
# 검증 및 테스트 데이터용 변환기
transform_test = A.Compose([
    A.Resize(450, 650),
    A.Normalize(),
    ToTensorV2()])

## 4.데이터셋 및 데이터 로더 생성

In [8]:
# 데이터셋 생성
img_dir = '/kaggle/input/plant-pathology-2020-fgvc7/images/'

dataset_train = ImageDataset(train, img_dir=img_dir, transform=transform_train)
dataset_valid = ImageDataset(valid, img_dir=img_dir, transform=transform_test)

In [9]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
g = torch.Generator()
g.manual_seed(0)

In [10]:
# 데이터 로더 생성
from torch.utils.data import DataLoader

batch_size = 4

loader_train = DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True,
                          worker_init_fn=seed_worker, generator=g, num_workers=2)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=batch_size, shuffle=False,
                          worker_init_fn=seed_worker, generator=g, num_workers=2)

# 모델 생성

In [11]:
!pip install efficientnet-pytorch==0.7.1

In [12]:
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=4)
model = model.to(device)

Loaded pretrained weights for efficientnet-b7


# 모델 훈련 및 성능 검증
## 1.손실함수와 옵티마이저 설정

In [13]:
import torch.nn as nn

# 손실함수
criterion = nn.CrossEntropyLoss()

In [14]:
# 옵티마이저
optimizer = torch.optim.AdamW(model.parameters(), lr=0.00006, weight_decay=0.0001)

## 2.훈련 및 성능 검증

In [15]:
from sklearn.metrics import roc_auc_score
from tqdm.notebook import tqdm
epochs = 3 # 에포크

for epoch in range(epochs):
    # 훈련
    model.train()
    epoch_train_loss = 0
    
    for images, labels in tqdm(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images) 
        loss = criterion(outputs, labels)
        epoch_train_loss += loss.item() 
        loss.backward()
        optimizer.step()
    print(f'에포크 [{epoch+1}/{epochs}] - 훈련 데이터 손실값: {epoch_train_loss/len(loader_train):.4f}')
    
    # 검증
    model.eval()
    epoch_valid_loss = 0
    preds_list = []
    true_onehot_list = []

    with torch.no_grad():
        for images, labels in loader_valid:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_valid_loss += loss.item()
            preds = torch.softmax(outputs.cpu(), dim=1).numpy()
            true_onehot = torch.eye(4)[labels.cpu()].numpy()
            preds_list.extend(preds)
            true_onehot_list.extend(true_onehot)
    print(f'에포크 [{epoch+1}/{epochs}] - 검증 데이터 손실값: {epoch_valid_loss/len(loader_valid):.4f}')
    print(f'검증 데이터 ROC AUC: {roc_auc_score(true_onehot_list, preds_list):.4f}')

  0%|          | 0/410 [00:00<?, ?it/s]

에포크 [1/3] - 훈련 데이터 손실값: 0.6911
에포크 [1/3] - 검증 데이터 손실값: 0.2782
검증 데이터 ROC AUC: 0.9463


  0%|          | 0/410 [00:00<?, ?it/s]

에포크 [2/3] - 훈련 데이터 손실값: 0.3480
에포크 [2/3] - 검증 데이터 손실값: 0.2004
검증 데이터 ROC AUC: 0.9710


  0%|          | 0/410 [00:00<?, ?it/s]

에포크 [3/3] - 훈련 데이터 손실값: 0.2499
에포크 [3/3] - 검증 데이터 손실값: 0.1646
검증 데이터 ROC AUC: 0.9704


# 예측 및 결과 제출

In [16]:
# 테스트 데이터 원본용 데이터셋 및 데이터 로더
dataset_test = ImageDataset(test, img_dir=img_dir, transform=transform_test, is_test=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False,
                         worker_init_fn=seed_worker, generator=g, num_workers=2)

## 1.예측

In [17]:
model.eval()
preds = np.zeros((len(test), 4))

with torch.no_grad():
    for i, images in enumerate(loader_test):
        images = images.to(device)
        outputs = model(images)
        preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy()
        preds[i*batch_size:(i+1)*batch_size] += preds_part

## 2.결과 제출

In [18]:
submission_test = submission.copy()
submission_test[['healthy', 'multiple_diseases', 'rust', 'scab']] = preds
submission_test.to_csv('submission_test.csv', index=False)

# 성능 개선 I
## 1.스케줄러

In [19]:
from transformers import get_cosine_schedule_with_warmup

epochs = 10
scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=len(loader_train)*3,
                                            num_training_steps=len(loader_train)*epochs)

## 2.훈련 및 성능 검증

In [20]:
for epoch in range(epochs):
    # 훈련
    model.train()
    epoch_train_loss = 0
    
    for images, labels in tqdm(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images) 
        loss = criterion(outputs, labels)
        epoch_train_loss += loss.item() 
        loss.backward()
        optimizer.step()
        scheduler.step()
    print(f'에포크 [{epoch+1}/{epochs}] - 훈련 데이터 손실값: {epoch_train_loss/len(loader_train):.4f}')
    
    # 검증
    model.eval()
    epoch_valid_loss = 0
    preds_list = []
    true_onehot_list = []

    with torch.no_grad():
        for images, labels in loader_valid:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            epoch_valid_loss += loss.item()
            preds = torch.softmax(outputs.cpu(), dim=1).numpy()
            true_onehot = torch.eye(4)[labels.cpu()].numpy()
            preds_list.extend(preds)
            true_onehot_list.extend(true_onehot)
    print(f'에포크 [{epoch+1}/{epochs}] - 검증 데이터 손실값: {epoch_valid_loss/len(loader_valid):.4f}')
    print(f'검증 데이터 ROC AUC: {roc_auc_score(true_onehot_list, preds_list):.4f}')

  0%|          | 0/410 [00:00<?, ?it/s]

에포크 [1/10] - 훈련 데이터 손실값: 0.1561
에포크 [1/10] - 검증 데이터 손실값: 0.1591
검증 데이터 ROC AUC: 0.9787


  0%|          | 0/410 [00:00<?, ?it/s]

에포크 [2/10] - 훈련 데이터 손실값: 0.1400
에포크 [2/10] - 검증 데이터 손실값: 0.1638
검증 데이터 ROC AUC: 0.9874


  0%|          | 0/410 [00:00<?, ?it/s]

...

에포크 [8/10] - 훈련 데이터 손실값: 0.0371
에포크 [8/10] - 검증 데이터 손실값: 0.1593
검증 데이터 ROC AUC: 0.9840


  0%|          | 0/410 [00:00<?, ?it/s]

에포크 [9/10] - 훈련 데이터 손실값: 0.0287
에포크 [9/10] - 검증 데이터 손실값: 0.1617
검증 데이터 ROC AUC: 0.9813


  0%|          | 0/410 [00:00<?, ?it/s]

에포크 [10/10] - 훈련 데이터 손실값: 0.0256
에포크 [10/10] - 검증 데이터 손실값: 0.1629
검증 데이터 ROC AUC: 0.9798


## 3.예측

In [21]:
# TTA용 데이터셋 및 데이터 로더
dataset_tta = ImageDataset(test, img_dir=img_dir, transform=transform_train, is_test=True)
loader_tta = DataLoader(dataset_tta, batch_size=batch_size, shuffle=False,
                        worker_init_fn=seed_worker, generator=g, num_workers=2)

In [22]:
num_tta = 5
preds_tta = np.zeros((len(test), 4))

for i in range(num_tta):
    with torch.no_grad():
        for j, images in enumerate(loader_tta):
            images = images.to(device)
            outputs = model(images)
            preds_part = torch.softmax(outputs.cpu(), dim=1).squeeze().numpy()
            preds_tta[i*batch_size:(i+1)*batch_size] += preds_part

In [23]:
preds_tta /= num_tta

## 4.결과 제출

In [24]:
submission_tta = submission.copy()
submission_tta[['healthy', 'multiple_diseases', 'rust', 'scab']] = preds_tta
submission_tta.to_csv('submission_tta.csv', index=False)

# 성능 개선 II
## 1.레이블 스무딩

In [25]:
def apply_label_smoothing(df, target, alpha, threshold):
    df_target = df[target].copy()
    k = len(target)
    
    for idx, row in df_target.iterrows():
        if (row > threshold).any():
            row = (1 - alpha)*row + alpha/k
            df_target.iloc[idx] = row
    return df_target

In [26]:
alpha = 0.001
threshold = 0.999
target = ['healthy', 'multiple_diseases', 'rust', 'scab']

## 2.결과 제출

In [27]:
submission_test_ls = submission_test.copy()
submission_test_ls[target] = apply_label_smoothing(submission_test_ls, target, alpha, threshold)
submission_test_ls.to_csv('submission_test_ls.csv', index=False)

In [28]:
submission_tta_ls = submission_tta.copy()
submission_tta_ls[target] = apply_label_smoothing(submission_tta_ls, target, alpha, threshold)
submission_tta_ls.to_csv('submission_tta_ls.csv', index=False)